In [1]:
import os
import matplotlib.pyplot as plt
import pandas
import torch
from torchvision import transforms
from transformers import ViTImageProcessor
import torch.nn as nn

2025-04-23 16:11:01.062164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745424661.315583      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745424661.385488      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Data preprocessing

In [2]:
path="/kaggle/input/skin-disease-dataset/skin-disease-datasaet"

In [3]:
train_path=os.path.join(path,"train_set")
test_path=os.path.join(path,"test_set")

In [4]:
os.listdir(train_path)

['FU-nail-fungus',
 'FU-ringworm',
 'VI-shingles',
 'BA-impetigo',
 'FU-athlete-foot',
 'VI-chickenpox',
 'PA-cutaneous-larva-migrans',
 'BA- cellulitis']

In [5]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms


In [6]:
class Skin_Dataset(Dataset):
    def __init__(self,train_folder,transform):
        self.annotations=[disease for disease in os.listdir(train_folder)]
        self.data=[]
        self.transform=transform
        self.class_to_idx={class_name:idx for idx,class_name in enumerate(self.annotations)}
        for cls in self.annotations:
            class_path=os.path.join(train_folder,cls)
            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                self.data.append((img_path, self.class_to_idx[cls]))
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label
    def help(self):
        print(self.class_to_idx)
        

In [7]:
transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

In [8]:
train_dataset = Skin_Dataset(train_path, transform=transform)
train_dataset.help()

{'FU-nail-fungus': 0, 'FU-ringworm': 1, 'VI-shingles': 2, 'BA-impetigo': 3, 'FU-athlete-foot': 4, 'VI-chickenpox': 5, 'PA-cutaneous-larva-migrans': 6, 'BA- cellulitis': 7}


In [9]:
print(len(train_dataset))

925


In [10]:
test_dataset = Skin_Dataset(test_path, transform=transform)


In [11]:
print(len(test_dataset))

234


In [12]:
from torch.utils.data import DataLoader
train_data_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_data_loader=DataLoader(test_dataset,batch_size=32,shuffle=False)

In [13]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
from transformers import ViTForImageClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=8  # we have 8 diseases
)

model.to(device)  
loss_fc=nn.CrossEntropyLoss()
import torch.optim as optim
optimizer=optim.Adam(model.parameters(),lr=5e-5)
num_epochs=10

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from PIL import Image

In [15]:
for epoch in range(num_epochs):
    model.train()
    for images,labels in train_data_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs=model(images)
        logits=outputs.logits
        loss=loss_fc(logits,labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 1/10, Loss: 1.1970
Epoch 2/10, Loss: 0.6059
Epoch 3/10, Loss: 0.2783
Epoch 4/10, Loss: 0.1956
Epoch 5/10, Loss: 0.1536
Epoch 6/10, Loss: 0.1309
Epoch 7/10, Loss: 0.1125
Epoch 8/10, Loss: 0.0972
Epoch 9/10, Loss: 0.0869
Epoch 10/10, Loss: 0.0779


tensor(0.0779, device='cuda:0', grad_fn=<NllLossBackward0>)


In [19]:
model.eval()
correct = 0
total = 0
val_loss = 0

with torch.no_grad():
    for images, labels in test_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)  
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs
        
        loss = loss_fc(logits, labels)
        val_loss += loss.item()
        
        _, predicted = torch.max(logits, dim=1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
avg_loss = val_loss / len(test_data_loader)
print(f"Validation Accuracy: {accuracy:.4f}, Validation Loss: {avg_loss:.4f}")


Validation Accuracy: 0.9786, Validation Loss: 0.1620


In [ ]:
#if wanna save it
torch.save(model.state_dict(),"vit_infections.pth")